Description: Code to query satellite data from Oceandata site and join with float/location data

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr
import os
import bisect
import calendar

In [4]:
#Define Appkey for oceandata query
#   An appkey can be obtained from:
#    https://oceandata.sci.gsfc.nasa.gov/appkey/
#Must sign up for Earthdata account first (free)

# appkey_val = "<ENTER APPKEY HERE>"
with open('../../../data/appkey.txt') as f:
    lines = f.readlines()
appkey_val = lines[0]
# appkey_val

In [3]:
#Read in profile data
float_loc_data = pd.read_csv("../../../data/floatLocData.csv")
float_loc_data.head()

,float,latitude,longitude,cycleNumber,date,date_str,PRES,PSAL,TEMP,BBP700,CHLA
0,1902303,49.236,-14.742,1,2021-05-06 02:03:16,2021-05-06 02:03:16,3,35.539001,12.378333,0.001788,0.695875
1,1902303,49.236,-14.742,1,2021-05-06 02:03:16,2021-05-06 02:03:16,4,35.539001,12.378333,0.001788,0.695875
2,1902303,49.236,-14.742,1,2021-05-06 02:03:16,2021-05-06 02:03:16,6,35.539001,12.378667,0.001788,0.696499
3,1902303,49.236,-14.742,1,2021-05-06 02:03:16,2021-05-06 02:03:16,8,35.539001,12.379000,0.001789,0.716489
4,1902303,49.236,-14.742,1,2021-05-06 02:03:16,2021-05-06 02:03:16,10,35.539001,12.379000,0.001790,0.715552


In [5]:
#Remove unneccesary columns and convert the datetime object for date to just a date
float_loc_data = float_loc_data.drop(['date_str'], axis=1)
float_loc_data['date'] = pd.to_datetime(float_loc_data['date'])
float_loc_data['short_date'] = float_loc_data['date'].dt.date

In [6]:
float_loc_data.head()

,float,latitude,longitude,cycleNumber,date,PRES,PSAL,TEMP,BBP700,CHLA,short_date
0,1902303,49.236,-14.742,1,2021-05-06 02:03:16,3,35.539001,12.378333,0.001788,0.695875,2021-05-06
1,1902303,49.236,-14.742,1,2021-05-06 02:03:16,4,35.539001,12.378333,0.001788,0.695875,2021-05-06
2,1902303,49.236,-14.742,1,2021-05-06 02:03:16,6,35.539001,12.378667,0.001788,0.696499,2021-05-06
3,1902303,49.236,-14.742,1,2021-05-06 02:03:16,8,35.539001,12.379000,0.001789,0.716489,2021-05-06
4,1902303,49.236,-14.742,1,2021-05-06 02:03:16,10,35.539001,12.379000,0.001790,0.715552,2021-05-06


In [7]:
#Determine how many unique dates exist in the float data (we will need to query each date)
dates = float_loc_data['short_date'].unique()
len(dates)

3871

In [8]:
#find the start date and end date for the merge
min_date = float_loc_data['short_date'].min()
max_date = float_loc_data['short_date'].max()
print(min_date)
print(max_date)

2010-06-23
2022-05-23


In [9]:
#Find the file locs for the 8 day measurements
# import datetime
# 8 day measurements always exist on January 1st for each year then appear again in 8 day increments 
from datetime import timedelta
date_values = []
for i in range(2010,2023):
    date_values_new = pd.date_range(start = datetime(i,1,1), end = datetime(i,12,31), freq='8D')
    date_values.extend(date_values_new)
#Sort so the bisect works when used later
date_values.sort()
date_values[0:4]

[Timestamp('2010-01-01 00:00:00', freq='8D'),
 Timestamp('2010-01-09 00:00:00', freq='8D'),
 Timestamp('2010-01-17 00:00:00', freq='8D'),
 Timestamp('2010-01-25 00:00:00', freq='8D')]

In [10]:
# lookup_date = dates[0]
# lookup_date

In [11]:
def query_data(filename, appkey_val, output_dir):
    """
    Description: Code to call the oceandata query

    Inputs: 
        filename: str, name of the file to pull from oceandata
        appkey_val: str, appkey from oceandata
        output_dir: str, location to store queried data
    
    Output:
        None (data stored in output directory)
    """

    current_folder = os.getcwd()
    command = "python oceandata_query.py " + filename +" --appkey "+appkey_val + " --odir " + current_folder + output_dir
    # print(command)
    os.system(command)


In [12]:
#now that we have the file, get values for lat/lon of interest
def lookup_sat_value(subset_df, filename, period, var_name, var_oceandata):
    """
    Description; Pull specific values of variable at given latitude and longitude

    Input:
        subset_df: pd.DataFrame, dataframe with latitude/longitude pairs
        filename: str, path to file with satellite data
        period: str, period of data pull {day, 8d, month}
        var_name: str, name of the variable of interest
        var_oceandata: str, name of variable on oceandata
    """
    i=0
    ds = xr.open_dataset(filename)
    var_value_name = var_oceandata.split('.')[1]
    for i in range(len(subset_df)):
        if not subset_df.iloc[i][var_name+"_"+period]:
            sat_val = ds.sel(lon=subset_df.iloc[i]['longitude'], lat=subset_df.iloc[i]['latitude'], method='nearest')
            if np.isnan(sat_val[var_value_name].values.item()):
                continue
            else:
                subset_df.iloc[i,subset_df.columns.get_loc(var_name+"_" + period)] = sat_val[var_value_name].values.item()
        else:
            continue
    return(subset_df)

In [13]:
# query_data('AQUA_MODIS.20220101.L3m.DAY.KD.Kd_490.4km.nc', appkey_val, '/temp/')
#AQUA_MODIS.20220101.L3m.DAY.POC.poc.4km.nc
#AQUA_MODIS.20220101.L3m.DAY.KD.Kd_490.4km.nc

In [16]:
# ds = xr.open_dataset('temp/AQUA_MODIS.20220101.L3m.DAY.KD.Kd_490.4km.nc')
# ds

In [17]:
def impute_period_col(subset_df, date_values, lookup_date, var_name, var_oceandata, period):
    """
    Description: Impute values for period into dataframe

    Inputs:
        subset_df: pd.DataFrame, dataframe for specific date
        date_values: list, list with all 8 day options
        lookup_date: datetime.date object, date of interest to pull in data for all locations
        var_name: str, name of column containing variable values to be added to dateframe
        var_oceandata: str, name of variable on oceandata
        period: str, period of interest {'day','8d','month'}

    Outputs:
        subset_df: pd.DataFrame, input dataframe for specific date with rows still containing missing values
        complete_subset: pd.DataFrame, dataframe for specific date with rows with filled values
    """
    if period == 'day':
        #Specify filename for daily data:
        # var_oceandata = 'CHL.chl_a'
        filename = "AQUA_MODIS."+str(lookup_date.year) + '{0:02d}'.format(lookup_date.month) + '{0:02d}'.format(lookup_date.day) \
            + ".L3m.DAY." + var_oceandata + ".4km.nc"
    elif period == '8d':
        #date list must be sorted to perform the bisection***
        #find day with the 8 day value file
        date_loc = bisect.bisect_right(date_values, pd.Timestamp(lookup_date))
        date_8d_start = date_values[date_loc-1]
        date_8d_end = date_values[date_loc] - timedelta(1)

        #Determine which file to open:
        filename = "AQUA_MODIS."+str(date_8d_start.year) + '{0:02d}'.format(date_8d_start.month) + '{0:02d}'.format(date_8d_start.day) \
            + "_" + str(date_8d_end.year) + '{0:02d}'.format(date_8d_end.month) + '{0:02d}'.format(date_8d_end.day)+".L3m.8D." + var_oceandata +".4km.nc"
    elif period == 'month':
        #monthly example:
        # AQUA_MODIS.20220101_20220131.L3m.MO.CHL.chlor_a.4km.nc 
        #Determine which file to open:
        filename = "AQUA_MODIS."+str(lookup_date.year) + '{0:02d}'.format(lookup_date.month) + '{0:02d}'.format(1) \
            + "_" + str(lookup_date.year) + '{0:02d}'.format(lookup_date.month) \
                + '{0:02d}'.format(calendar.monthrange(lookup_date.year, lookup_date.month)[1])+".L3m.MO."+var_oceandata+".4km.nc"
    # print(filename)
    #Query data
    query_data(filename, appkey_val, '/temp/')
    #Add columns for variable of interest if not already there
    if var_name+"_"+period not in subset_df:
        subset_df[[var_name+"_"+period]] = None
    #Read in values from queried data
    try:
        subset_df = lookup_sat_value(subset_df, './temp/'+filename, period, var_name, var_oceandata)
        #delete file to save space
        os.system("rm ./temp/" + filename)
    except: 
        print(filename, "did not download")

    return(subset_df)
    

In [18]:
def impute_for_date(df, date_values, lookup_date, appkey_val, var_name, var_oceandata):
    """
    Description: Pull values from oceandata for a specific date for all lat/lon pairs in float data

    Inputs:
        df: pd.DataFrame, full float data dataframe
        date_values: list, list with all 8 day options
        lookup_date: datetime.date object, date of interest to pull in data for all locations
        appkey_val: str, appkey for oceandata
        var_name: str, name of column containing variable values to be added to dateframe
        var_oceandata: str, name of variable on oceandata

    Outputs:
        df: pd.DataFrame, float data dataframe with values imputed for specified lookup_date
    """
    #Subset the full float data
    subset_df = df[df['short_date'] == lookup_date]
    #Keep only unique pairs of lat/lon for specified date
    subset_df = subset_df[['latitude','longitude']]
    subset_df = subset_df.drop_duplicates()
    # subset_df = subset_df.reset_index()

    #Query data fore each period and impute missing values
    subset_df = impute_period_col(subset_df, date_values, lookup_date, var_name, var_oceandata, 'day')
    subset_df = impute_period_col(subset_df, date_values, lookup_date, var_name, var_oceandata,'8d')
    subset_df = impute_period_col(subset_df, date_values, lookup_date, var_name, var_oceandata,'month')
    subset_df['short_date'] = lookup_date

    return(subset_df)

In [20]:
#To find available variables use: 
# https://oceancolor.gsfc.nasa.gov/docs/format/Ocean_Data_Product_Users_Guide.pdf
    # Guide with descriptions of variables
# https://oceandata.sci.gsfc.nasa.gov/directdataaccess/Level-3%20Mapped/Aqua-MODIS/2022/001/ 
    # Actual data example names (where we are pulling hte data form)
#Takes ~350 minutes per variable
#CHL.chlor_a: Chlorophyll Concentration, OCI Algorithm
# FLH.ipar: Instantaneous Photosynthetically Available Radiation
# POC.poc: Particulate Organic Carbon, D. Stramski, 2007 (443/555 version)
# PIC.pic: Calcite Concentration, Balch and Gordon
# KD.Kd_490: Diffuse attenuation coefficient at 490 nm, KD2 algorithm
# FLH.nflh: Normalized Fluorescence Line Height
# IOP.adg_443: Absorption due to gelbstoff and detrital material at 443nm, GIOP model
# IOP.adg_s: Detrital and gelbstoff absorption spectral parameter for GIOP model
# IOP.adg_unc_443: Uncertainty in absorption due to gelbstoff and detrital material at 443 nm, GIOP model
# IOP.aph_443: 
# IOP.aph_unc_443: 
# IOP.a_678: 
# IOP.bbp_443:
# IOP.bbp_s:
# IOP.bbp_unc_443:
# IOP.bb_678:
# NSST.sst:
# SST.sst:
# SST4.sst4: 
# PAR.par:
# RRS.angstrom: 
# RRS.aot_869:
# RRS.Rrs_678:
# 23 variables * 3.5MB = 80MB
oceandata_variables = ['CHL.chlor_a','FLH.ipar', 'POC.poc','PIC.pic','KD.Kd_490']
filled_df = pd.DataFrame(columns = ['latitude','longitude','sat_kd_490_day','sat_kd_490_8d','sat_kd_490_month','short_date'])
j = 387.1
for i in range(0,len(dates)):
    if i>= j:
        print("~", round(i/len(dates)*100,0),"% complete")
        j = j+387.1
    new_subset = impute_for_date(float_loc_data, date_values, dates[i], appkey_val, 'sat_kd_490','KD.Kd_490')
    filled_df = pd.concat([filled_df, new_subset])

There was a problem retrieving AQUA_MODIS.20220406.L3m.DAY.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220406.L3m.DAY.KD.Kd_490.4km.nc did not download
There was a problem retrieving AQUA_MODIS.20220415_20220422.L3m.8D.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220415_20220422.L3m.8D.KD.Kd_490.4km.nc did not download
There was a problem retrieving AQUA_MODIS.20220404.L3m.DAY.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220404.L3m.DAY.KD.Kd_490.4km.nc did not download
There was a problem retrieving AQUA_MODIS.20220408.L3m.DAY.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220408.L3m.DAY.KD.Kd_490.4km.nc did not download
There was a problem retrieving AQUA_MODIS.20220415_20220422.L3m.8D.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220415_20220422.L3m.8D.KD.Kd_490.4km.nc did not download
There was a problem retrieving AQUA_MODIS.20220407.L3m.DAY.KD.Kd_490.4km.nc (received status 404)
AQUA_MODIS.20220407.L3m.DAY.KD.Kd_490.4km.nc did not download
There wa

In [104]:
#If the data pull is chunked, concatenate into one file
# full_df_final = pd.concat([full_df2, filled_df_4])

In [20]:
#Save data to csv
filled_df.to_csv('../../../data/sat_kd_490_full_final.csv', index=False)

In [2]:
#Read in profile data
# float_data_full = pd.read_csv("../../../data/float_loc_data_new_vars.csv")
# float_data_full['date'] = pd.to_datetime(float_data_full['date'])
# float_data_full['short_date'] = float_data_full['date'].dt.date
# float_data_full.head()

In [8]:
# pic_data  = pd.read_csv("../../../data/sat_pic_full_final.csv")
# pic_data['short_date'] = pd.to_datetime(pic_data['short_date'])
# pic_data['short_date'] = pic_data['short_date'].dt.date
# pic_data.head()

In [9]:
# float_data_full2 = float_data_full.merge(pic_data, how='left',on=['latitude','longitude','short_date'])
# float_data_full2.head()
# float_data_full2.to_csv('../../../data/float_loc_data_new_vars2.csv', index=False)

In [51]:
#12532828	-54.455	-136.128
#from this test we see that the MO_SAT_CHL data was missing in the original file too
test = float_loc_data.iloc[11239158]
print(test)
satData = pd.read_csv("../../../data/satData.csv")
satData[(satData['float'] == 5905371)&(satData['cycleNumber'] == 120) ]

float                      5905371
latitude                   -54.455
longitude                 -136.128
cycleNumber                    120
date           2021-05-06 10:32:26
PRES                             4
PSAL                     33.992001
TEMP                      5.323282
BBP700                    0.000695
CHLA                        0.3588
short_date              2021-05-06
Name: 11239158, dtype: object


,float,cycleNumber,date,latitude,longitude,elevation,MO_SAT_SST,MO_SAT_CHL,MO_SAT_BBP,LT_SAT_SST_SD,LT_SAT_SST_MED,LT_SAT_CHL_SD,LT_SAT_CHL_MED,LT_SAT_BBP_SD,LT_SAT_BBP_MED
25487,5905371,120,2021-05-06 10:32:26,-54.455,-136.128,-2716.0,5.465739,NaN,NaN,1.204847,4.497638,0.128591,0.1544,0.00159,0.002506
